**Parallel Programming in Python**

**hello_with_threads.py**



In [ ]:
from threading import Thread
from time import sleep

class CookBook(Thread):
	def __init__(self):
		Thread.__init__(self)
		self.message = "Hello from CookBook!\n"

	def print_message(self):
		print(self.message)

	def run(self):
		print("Thread starting\n")
		x = 0
		while x < 10:
			self.print_message()
			sleep(2)
			x += 1

		print("Thread ended\n")

print("Process started")

cook_book = CookBook()
cook_book.start()

print("Process ended")

Process started
Thread starting
Process ended

Hello from CookBook!



**define_a_thread.py**

In [ ]:
import threading
from time import sleep

def my_function(i):
	for j in range(3):
		print("Function called by thread ", i, "\n")
		sleep(1)

threads = []

for i in range(5):
	# note that we need the , in the args tuple otherwise we get
	# some ugly errors.
	t = threading.Thread(target=my_function, args=(i,))
	threads.append(t)
	# This will actually start the thread
	t.start()
	# This will force the main thread to wait until the called thread
	# is done. Comment this out and see how the behavior changes
	t.join()

Function called by thread  0 

Function called by thread  0 

Function called by thread  0 

Function called by thread  1 

Function called by thread  1 

Function called by thread  1 

Function called by thread  2 

Function called by thread  2 

Function called by thread  2 

Function called by thread  3 

Function called by thread  3 

Function called by thread  3 

Function called by thread  4 

Function called by thread  4 

Function called by thread  4 



**determine_current_thread.py **

In [ ]:
import threading
import time

def thread_function():
	name = threading.currentThread().getName()
	print(name + ' is starting\n')
	time.sleep(2)
	print(name + ' is exiting\n')

# for i in range(5):
# 	t = threading.Thread(name=str(i), target=thread_function)
# 	t.start()
# 	#t.join()

t1 = threading.Thread(name='Function 1', target=thread_function)
t2 = threading.Thread(target=thread_function)

t1.start()
t2.start()

t1.join()
t2.join()

Function 1 is starting

Thread-10 is starting

Function 1 is exiting

Thread-10 is exiting



**thread_synchronization_with_lock**

In [ ]:
import threading

COUNT = 100000

shared_resource_with_lock  = 0
shared_resource_with_no_lock = 0

shared_resource_lock = threading.Lock()

def increment_with_lock():
	global shared_resource_with_lock
	for i in range(COUNT):
		shared_resource_lock.acquire()
		shared_resource_with_lock += 1
		shared_resource_lock.release()

def decrement_with_lock():
	global shared_resource_with_lock
	for i in range(COUNT):
		shared_resource_lock.acquire()
		shared_resource_with_lock -= 1
		shared_resource_lock.release()

def increment_without_lock():
	global shared_resource_with_no_lock
	for i in range(COUNT):
		shared_resource_with_no_lock += 1

def decrement_without_lock():
	global shared_resource_with_no_lock
	for i in range(COUNT):
		shared_resource_with_no_lock -= 1

# LOCK MANAGEMENT DEMO
t1 = threading.Thread(target=increment_with_lock)
t2 = threading.Thread(target=decrement_with_lock)

t1.start()
t2.start()

t1.join()
t2.join()

print('The value of the resource with lock management is ', shared_resource_with_lock)

# WITHOUT LOCK MANAGEMENT DEMO
t3 = threading.Thread(target=increment_without_lock)
t4 = threading.Thread(target=decrement_without_lock)

t3.start()
t4.start()

t3.join()
t4.join()

print('The value of the resouce without lock management is ', shared_resource_with_no_lock)

The value of the resource with lock management is  0
The value of the resouce without lock management is  0


**thread_synchronization_with_rlock**

In [ ]:
import threading
import time

class Box(object):
	lock = threading.RLock()

	def __init__(self):
		self.total_items = 0

	def execute(self, n):
		Box.lock.acquire()
		self.total_items += n
		Box.lock.release()

	def add(self):
		Box.lock.acquire()
		self.execute(1)
		Box.lock.release()

	def remove(self):
		Box.lock.acquire()
		self.execute(-1)
		Box.lock.release()

# Run these functions in seperate threads
# and call the Box's methods

def adder(box, items):
	while items > 0:
		print("Adding 1 item in the box\n")
		box.add()
		time.sleep(2)
		items -= 1

def remover(box, items):
	while items > 0:
		print("Removing 1 item from the box\n")
		box.remove()
		time.sleep(2)
		items -= 1


items = 5
box = Box()

print('Putting %s items in the box' % items)

t1 = threading.Thread(target=adder, args=(box, items))
t2 = threading.Thread(target=remover, args=(box, items))

t1.start()
t2.start()

t1.join()
t2.join()

print('%s items still remain in the box' % box.total_items)

Putting 5 items in the box
Adding 1 item in the box

Removing 1 item from the box

Adding 1 item in the box

Removing 1 item from the box

Adding 1 item in the box

Removing 1 item from the box

Adding 1 item in the box
Removing 1 item from the box


Removing 1 item from the box

Adding 1 item in the box

0 items still remain in the box


**Thread_synch_with_semaphores/consumer_producer.py**

In [ ]:
import threading
import time
import random

semaphore = threading.Semaphore(0)

def consumer():
	print("Consumer is waiting")
	semaphore.acquire()
	print("Consumer notify: consumed item number %s" %item)

def producer():
	global item
	time.sleep(10)
	item = random.randint(0, 1000)
	print("Producer notify : produced item number %s" % item)

	semaphore.release()

for i in range(5):
	t1 = threading.Thread(target=producer)
	t2 = threading.Thread(target=consumer)

	t1.start()
	t2.start()

	t1.join()
	t2.join()

Consumer is waiting
Producer notify : produced item number 791
Consumer notify: consumed item number 791
Consumer is waiting
Producer notify : produced item number 113
Consumer notify: consumed item number 113
Consumer is waiting
Producer notify : produced item number 594
Consumer notify: consumed item number 594
Consumer is waiting
Producer notify : produced item number 559
Consumer notify: consumed item number 559
Consumer is waiting
Producer notify : produced item number 97
Consumer notify: consumed item number 97


**Thread_communication_with_queue/queue_demo.py**

In [ ]:
from threading import Thread, Event
from queue import Queue
import time
import random

class producer(Thread):
	def __init__(self, queue):
		Thread.__init__(self)
		self.queue = queue

	def run(self):
		for i in range(10):
			item = random.randint(0, 256)

			self.queue.put(item)
			print('Producer notify : item %d appended to queue by %s \n' % (item, self.name))

			time.sleep(1)

class consumer(Thread):
	def __init__(self, queue):
		Thread.__init__(self)
		self.queue = queue

	def run(self):
		while True:
			item = self.queue.get()
			print('Consumer notify : %d popped from queue by %s \n' % (item, self.name))
			self.queue.task_done()

queue = Queue()
t1 = producer(queue)
t2 = consumer(queue)
t3 = consumer(queue)

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

Producer notify : item 22 appended to queue by Thread-15 

Consumer notify : 22 popped from queue by Thread-16 

Producer notify : item 8 appended to queue by Thread-15 

Consumer notify : 8 popped from queue by Thread-16 

Producer notify : item 66 appended to queue by Thread-15 

Consumer notify : 66 popped from queue by Thread-17 

Producer notify : item 109 appended to queue by Thread-15 

Consumer notify : 109 popped from queue by Thread-16 

Producer notify : item 72 appended to queue by Thread-15 

Consumer notify : 72 popped from queue by Thread-17 

Producer notify : item 183 appended to queue by Thread-15 

Consumer notify : 183 popped from queue by Thread-16 

Producer notify : item 48 appended to queue by Thread-15 

Consumer notify : 48 popped from queue by Thread-17 

Producer notify : item 130 appended to queue by Thread-15 

Consumer notify : 130 popped from queue by Thread-16 

Producer notify : item 94 appended to queue by Thread-15 

Consumer notify : 94 popped from 

**spawning_a_process/spawn_a_process.py**

In [ ]:
import multiprocessing
import time

def function(i):
	print('Called function in process %s' % i)
	time.sleep(10)

process_jobs = []

for i in range(5):
	p = multiprocessing.Process(target=function, args=(i,))
	process_jobs.append(p)
	p.start()
	p.join()

**Naming_a_process/naming_a_process.py**

In [ ]:
import multiprocessing
import time

def foo():
	name = multiprocessing.current_process().name
	print('Starting %s\n' % name)
	time.sleep(3)
	print('Exiting %s\n' % name)

process_with_name = multiprocessing.Process(name='foo_process', target=foo)
process_with_name.start()
process_with_name.join()

Starting foo_process

Exiting foo_process



**Background_process**

In [ ]:
import multiprocessing
import time

def foo():
	name = multiprocessing.current_process().name
	print('Starting %s\n' % name)
	time.sleep(10)
	print('Exiting %s\n' % name)

process_with_name = multiprocessing.Process(name='foo_process', target=foo)
# Will make it a background process
process_with_name.daemon = True
process_with_name.start()

Starting foo_process



**Kill_a_process**

In [ ]:
import multiprocessing
import time

def foo():
	print('Starting function')
	time.sleep(0.1)
	print('Finished function')

p = multiprocessing.Process(target=foo)
print('Process before execution: ', p, p.is_alive())

p.start()
print('Process running: ', p, p.is_alive())

p.terminate()
print('Process terminated: ', p, p.is_alive())

p.join()
print('Process joined: ', p, p.is_alive())

print('Process exit code: ', p.exitcode)

Process before execution:  <Process(Process-10, initial)> False
Process running:  <Process(Process-10, started)> True
Process terminated:  <Process(Process-10, started)> True
Process joined:  <Process(Process-10, stopped[SIGTERM])> False
Process exit code:  -15


**Exchange_objects_between_processes/using_pipes.py**

In [ ]:
import multiprocessing

def create_items(pipe):
	output_pipe, _ = pipe

	for i in range(10):
		output_pipe.send(i)

	output_pipe.close()

def multiply_items(pipe_1, pipe_2):
	close, input_pipe = pipe_1
	close.close()
	output_pipe, _ = pipe_2

	try:
		while True:
			item = input_pipe.recv()
			output_pipe.send(item * item)
	except EOFError:
		output_pipe.close()

pipe_1 = multiprocessing.Pipe(True)
process_pipe_1 = multiprocessing.Process(target=create_items, args=(pipe_1,))
process_pipe_1.start()

pipe_2 = multiprocessing.Pipe(True)
process_pipe_2 = multiprocessing.Process(target=multiply_items, args=(pipe_1, pipe_2))
process_pipe_2.start()

pipe_1[0].close()
pipe_2[0].close()

try:
	while True:
		print(pipe_2[1].recv())
except EOFError:
	print("End")

0
1
4
9
16
25
36
49
64
81
End


**Concurrent_futures/pooling_with_concurrent_futures.py**

In [ ]:
import concurrent.futures
import time

number_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def evaluate_item(x):
	result = count(x)
	print("item " + str(x) + " result: " + str(result))

def count(number):
	for i in range(0, 10000000):
		i = i + 1
	return i * number

# Evaluate sequentially
start_time = time.clock()

for item in number_list:
	evaluate_item(item)

print("Sequential execution in " + \
	  str(time.clock() - start_time) + " seconds")


# Evaluate with thread pool execution
start_time = time.clock()

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
	for item in number_list:
		executor.submit(evaluate_item, item)

print("Thread pool execution in " + \
	  str(time.clock() - start_time) + " seconds")


# Evaluate with process pool execution
start_time = time.clock()

with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
	for item in number_list:
		executor.submit(evaluate_item, item)

print("Process pool execution in " + \
	  str(time.clock() - start_time) + " seconds")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()


item 1 result: 10000000
item 2 result: 20000000
item 3 result: 30000000
item 4 result: 40000000
item 5 result: 50000000
item 6 result: 60000000
item 7 result: 70000000
item 8 result: 80000000
item 9 result: 90000000
item 10 result: 100000000
Sequential execution in 4.788508 seconds


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


item 2 result: 20000000
item 1 result: 10000000
item 4 result: 40000000
item 3 result: 30000000
item 5 result: 50000000
item 7 result: 70000000
item 9 result: 90000000
item 8 result: 80000000
item 6 result: 60000000
item 10 result: 100000000
Thread pool execution in 5.211219 seconds


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


item 1 result: 10000000
item 2 result: 20000000
item 5 result: 50000000
item 3 result: 30000000
item 4 result: 40000000
item 7 result: 70000000
item 6 result: 60000000
item 8 result: 80000000
item 10 result: 100000000
item 9 result: 90000000
Process pool execution in 0.06610099999999974 seconds


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


**Process_pool/process_pool.py**

In [ ]:
import multiprocessing

def square(data):
	return data * data

inputs = list(range(0, 100))
pool = multiprocessing.Pool(processes=4)
pool_outputs = pool.map(square, inputs)

pool.close()
pool.join()

print('Pool		:', pool_outputs)

Pool		: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


**Parallel program using Threads in Python**

In [ ]:
from threading import Thread,current_thread
print(current_thread().getName())
def mt():
 print("Child Thread")
 for i in range(11,20):
  print(i*2)
def disp():
 for i in range(10):
  print(i*2)
child=Thread(target=mt)
child.start()
disp()
print("Executing thread name :",current_thread().getName())
from threading import Thread,current_thread
class mythread(Thread):
 def run(self):
  for x in range(7):
   print("Hi from child")
a = mythread()
a.start()
a.join()
print("Bye from",current_thread().getName())

MainThread
Child Thread0
2
4
6
8
10
12
14
16
18
Executing thread name : MainThread

22
24
26
28
30
32
34
36
38
Hi from child
Hi from child
Hi from child
Hi from child
Hi from child
Hi from child
Hi from child
Bye from MainThread


**Parallel program using Process in Python**

In [ ]:
import multiprocessing
def worker(num):
 print('Worker:', num)
 for i in range(num):
   print(i)
 return
jobs = []
for i in range(1,5):
 p = multiprocessing.Process(target=worker, args=(i+10,))
 jobs.append(p)
 p.start()

Worker: 11
Worker: 12
Worker: 13
0
1
0
1
2
2
3
0
3
4
1
2
5
3
6
7
4
8
4
6
9
5
Worker: 14
7
10
8
11
0
5
12
9
6
1
10
7
8
9
2
10
3
11
4
5
6
7
8
9
10
11
12
13


In [ ]:
import multiprocessing

def worker(num):
    """thread worker function"""
    print ('Worker:', num)
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,))
        jobs.append(p)
        p.start()

Worker: 0
Worker: 2
Worker: 1
Worker: 3
Worker: 4


**Race Condition**

In [ ]:
import threading
x = 0 # A shared value
COUNT = 100
def incr():
 global x
 for i in range(COUNT):
  x += 1
  print(x)
def decr():
 global x
 for i in range(COUNT):
  x -= 1
  print(x)
t1 = threading.Thread(target=incr)
t2 = threading.Thread(target=decr)
t1.start()
t2.start()
t1.join()
t2.join()
print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0


**Synchronization in Python using Lock**

In [1]:
import threading
x = 0 # A shared value
COUNT = 100
lock = threading.Lock()
def incr():
 global x
 lock.acquire()
 print("thread locked for increment cur x=",x)
 for i in range(COUNT):
  x += 1
  print(x)
 lock.release()
 print("thread release from increment cur x=",x)
def decr():
 global x
 lock.acquire()
 print("thread locked for decrement cur x=",x)
 for i in range(COUNT):
  x -= 1
  print(x)
 lock.release()
 print("thread release from decrement cur x=",x)
t1 = threading.Thread(target=incr)
t2 = threading.Thread(target=decr)
t1.start()
t2.start()
t1.join()
t2.join()

thread locked for increment cur x= 0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
thread release from increment cur x= 100
thread locked for decrement cur x= 100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
thread release from decrement cur x= 0


**Synchronization in Python using Semaphore**

In [2]:
import threading
import time
done = threading.Semaphore(0)
item = None
def producer():
 global item
 print ("I'm the producer and I produce data.")
 print ("Producer is going to sleep.")
 time.sleep(10)
 item = "Hello"
 print ("Producer is alive. Signaling the consumer.")
 done.release()
def consumer():
 print ("I'm a consumer and I wait for data.")
 print ("Consumer is waiting.")
 done.acquire()
 print ("Consumer got", item)
t1 = threading.Thread(target=producer)
t2 = threading.Thread(target=consumer)
t1.start()
t2.start()

I'm the producer and I produce data.
Producer is going to sleep.
I'm a consumer and I wait for data.
Consumer is waiting.


**Synchronization in Python using event**

In [4]:
import threading
import time
item = None
# A semaphore to indicate that an item is available
available = threading.Semaphore(0)
# An event to indicate that processing is complete
completed = threading.Event()
# A worker thread
def worker():
 while True:
  available.acquire()
  print ("worker: processing", item)
 time.sleep(5)
 print ("worker: done")
 completed.set()

# A producer thread
def producer():
 global item
 for x in range(5):
  completed.clear() # Clear the event
  item = x # Set the item
  print ("producer: produced an item")
 available.release() # Signal on the semaphore
 completed.wait()
 print ("producer: item was processed")
t1 = threading.Thread(target=producer)
t1.start()
t2 = threading.Thread(target=worker)
t2.setDaemon(True)
t2.start()

producer: produced an item
producer: produced an item
producer: produced an item
producer: produced an item
producer: produced an item
worker: processing 4


**Producer and Consumer problem using thread**

In [ ]:
import threading, time, queue
items = queue.Queue()
# A producer thread
def producer():
 print ("I'm the producer")
 for i in range(30):
  items.put(i)
  time.sleep(1)
# A consumer thread
def consumer():
 print ("I'm a consumer", threading.currentThread().name)
 while True:
  x = items.get()
  print (threading.currentThread().name,"got", x)
 time.sleep(5)
# Launch a bunch of consumers
cons = [threading.Thread(target=consumer)
 for i in range(10)]
for c in cons:
 c.setDaemon(True)
 c.start()
# Run the producer
producer()

I'm a consumer Thread-20
I'm a consumer Thread-21
I'm a consumer Thread-22I'm a consumer
 Thread-23
I'm a consumerI'm a consumer Thread-25
 Thread-24
I'm a consumer Thread-26
I'm a consumer I'm a consumer Thread-28
Thread-27I'm a consumer Thread-29

I'm the producer
Thread-20 got 0
Thread-21 got 1
Thread-23 got 2
Thread-22 got 3
Thread-25 got 4
Thread-24 got 5
Thread-26 got 6
Thread-28 got 7
Thread-29 got 8
Thread-27 got 9
Thread-20 got 10
Thread-21 got 11
Thread-23 got 12
Thread-22 got 13
Thread-25 got 14
Thread-24 got 15
Thread-26 got 16
Thread-28 got 17
Thread-29 got 18
Thread-27 got 19
Thread-20 got 20
Thread-21 got 21
Thread-23 got 22
Thread-22 got 23
Thread-25 got 24
Thread-24 got 25
Thread-26 got 26
Thread-28 got 27
Thread-29 got 28
Thread-27 got 29


**Producer and Consumer problem using thread**

In [ ]:
import threading
import time
# A list of items that are being produced. Note: it is actually
# more efficient to use a collections.deque() object for this.
items = []
# A condition variable for items
items_cv = threading.Condition()
def producer():
 print ("I'm the producer")
 for i in range(30):
  with items_cv: # Always must acquire the lock first
   items.append(i) # Add an item to the list
   items_cv.notify() # Send a notification signal
  time.sleep(1)
def consumer():
 print ("I'm a consumer", threading.currentThread().name)
 while True:
  with items_cv: # Must always acquire the lock
   while not items: # Check if there are any items
    items_cv.wait() # If not, we have to sleep
    x = items.pop(0) # Pop an item off
 print (threading.currentThread().name,"got", x)
 time.sleep(5)
cons = [threading.Thread(target=consumer)
 for i in range(10)]
for c in cons:
 c.setDaemon(True)
 c.start()
producer()

I'm a consumer Thread-30I'm a consumer
I'm a consumer Thread-32
 Thread-31
I'm a consumer Thread-33
I'm a consumer I'm a consumer Thread-35
Thread-34
I'm a consumer Thread-36
I'm a consumer Thread-37
I'm a consumer Thread-38
I'm a consumer Thread-39
I'm the producer


In [ ]:
import multiprocessing as mp
from sympy import *
p=mp.Pool(processes=3)
x = symbols('x')
print(p.map(integrate,[x,x**2,x**3]))

[x**2/2, x**3/3, x**4/4]


In [ ]:
import numpy as np
import multiprocessing as mp
import numpy.random as npr
from sympy import *
def gsrm(i):
 n=200
 A = npr.randn(n,n)
 ev=np.linalg.eigvals(A)
 return i,ev[0].real
p=mp.Pool(processes=4)
print(p.map(gsrm,range(9)))
results=p.imap_unordered(gsrm,range(9))
for r in results:
 print (r)


[(0, -2.0410286158583095), (1, -2.0410286158583095), (2, -2.0410286158583095), (3, -2.0410286158583095), (4, 15.507669022387468), (5, 15.507669022387468), (6, 15.507669022387468), (7, 15.507669022387468), (8, -0.5795264376068983)]
(2, -0.5795264376068983)
(0, -0.5795264376068983)
(1, -0.5795264376068983)
(3, -11.447483238505804)
(4, -11.447483238505804)
(5, -11.447483238505804)
(6, -11.447483238505804)
(7, 13.724096078481171)
(8, 13.724096078481171)


In [ ]:
import os, math
from multiprocessing import Process
A=[0,0,0,0]
def my_function(i):
 A[i]=A[i]+1
 return
list_of_procs = []
for r in range(4):
 p = Process(target=my_function, args=(r,))
list_of_procs.append(p)
p.start()
for p in list_of_procs:
 p.join()
print(A)


[0, 0, 0, 0]


In [ ]:
import threading
import time


def print_work_a():
    print('Starting of thread :', threading.currentThread().name)
    time.sleep(2)
    print('Finishing of thread :', threading.currentThread().name)


def print_work_b():
    print('Starting of thread :', threading.currentThread().name)
    print('Finishing of thread :', threading.currentThread().name)

a = threading.Thread(target=print_work_a, name='Thread-a', daemon=True)
b = threading.Thread(target=print_work_b, name='Thread-b')

a.start()
b.start()

Starting of thread : Thread-a
Starting of thread : Thread-b
Finishing of thread : Thread-b
